#  Notebook : `01b_exploration_data.ipynb`

### **Objectif**

  - Explorer le dataset hybride avant la vectorisation
  - Identifier les tendances, corrélations et biais
  - Guider les choix de features et modèles pour la suite

---

In [ ]:
# ================================================================
# 🧠 1. Imports & setup
# ================================================================


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8,5)

DATA_PROCESSED = "../../data/processed/"
REPORTS_DIR = "../../reports/"
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(os.path.join(REPORTS_DIR, "figures"), exist_ok=True)

# Chargement du dataset réduit et des interactions
df = pd.read_csv(os.path.join(DATA_PROCESSED, "train_hybride_reduced.csv"))
print(f"✅ Dataset chargé : {df.shape[0]:,} lignes, {df.shape[1]} colonnes")

# Optionnel : si user_favs existe déjà
user_favs_path = os.path.join(DATA_PROCESSED, "user_favs_temp.csv")
user_favs = pd.read_csv(user_favs_path) if os.path.exists(user_favs_path) else None
if user_favs is not None:
    print(f"Interractions utilisateur chargées : {user_favs.shape[0]:,} lignes")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


✅ Dataset chargé : 1,462,009 lignes, 16 colonnes


In [ ]:
# ================================================================
# 📊 2. Vue d’ensemble des données
# ================================================================
display(df.head(3))
print("\n--- INFO ---")
print(df.info())
print("\n--- DESCRIPTION NUMÉRIQUE ---")
display(df.describe().T)

print(f"Utilisateurs uniques : {df['username'].nunique()}")
print(f"Animes uniques : {df['mal_id'].nunique()}")


,mal_id,title,genres,themes,type,rating,score,members,popularity,year,synopsis_length,completion_rate,drop_rate,weighted_score,total_votes,username
0,45649,The First Slam Dunk,"['Award Winning', 'Sports']","['School', 'Team Sports']",Movie,PG-13 - Teens 13 or older,8.72,69803,2881,0.0,113,0.555468,0.006759,8.668793,32134.0,ishikawas
1,38680,Fruits Basket 1st Season,"['Drama', 'Romance', 'Supernatural']","['Love Polygon', 'School']",TV,PG-13 - Teens 13 or older,8.21,916323,211,2019.0,213,0.569366,0.039585,8.165223,435739.0,ishikawas
2,795,Oniisama e...,"['Drama', 'Girls Love']","['Psychological', 'School']",TV,PG-13 - Teens 13 or older,7.86,53824,3315,1991.0,176,0.249485,0.031223,7.837709,11960.0,ishikawas



--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462009 entries, 0 to 1462008
Data columns (total 16 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   mal_id           1462009 non-null  int64  
 1   title            1462009 non-null  object 
 2   genres           1462009 non-null  object 
 3   themes           1462009 non-null  object 
 4   type             1462009 non-null  object 
 5   rating           1462009 non-null  object 
 6   score            1462009 non-null  float64
 7   members          1462009 non-null  int64  
 8   popularity       1462009 non-null  int64  
 9   year             1462009 non-null  float64
 10  synopsis_length  1462009 non-null  int64  
 11  completion_rate  1462009 non-null  float64
 12  drop_rate        1462009 non-null  float64
 13  weighted_score   1462009 non-null  float64
 14  total_votes      1462009 non-null  float64
 15  username         1462009 non-null  object 
dtypes: f

,count,mean,std,min,25%,50%,75%,max
mal_id,1462009.0,2.097673e+04,1.780084e+04,1.0,2890.000000,1.707400e+04,3.664900e+04,6.256800e+04
score,1462009.0,8.235520e+00,6.536610e-01,0.0,7.920000,8.310000e+00,8.640000e+00,9.290000e+00
members,1462009.0,1.354196e+06,1.030303e+06,23228.0,499878.000000,1.121020e+06,1.977969e+06,4.230312e+06
popularity,1462009.0,4.483562e+02,7.419353e+02,1.0,44.000000,1.480000e+02,4.970000e+02,5.006000e+03
year,1462009.0,1.760092e+03,6.662280e+02,0.0,2005.000000,2.012000e+03,2.017000e+03,2.026000e+03
synopsis_length,1462009.0,1.631697e+02,3.505700e+01,0.0,140.000000,1.650000e+02,1.830000e+02,3.700000e+02
completion_rate,1462009.0,6.378789e-01,1.724834e-01,0.0,0.554265,6.645215e-01,7.585827e-01,9.672530e-01
drop_rate,1462009.0,2.795773e-02,2.121955e-02,0.0,0.014840,2.353282e-02,3.471266e-02,3.805747e-01
weighted_score,1462009.0,8.205439e+00,5.874769e-01,0.0,7.879650,8.278976e+00,8.611927e+00,1.000000e+01
total_votes,1462009.0,8.050694e+05,6.996501e+05,0.0,240910.000000,6.323980e+05,1.221747e+06,2.980061e+06


Utilisateurs uniques : 192174
Animes uniques : 4828


In [ ]:


import matplotlib
# set a non-interactive backend to avoid backend_agg import issues on some environments
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns

# ================================================================
# 🔍 3. Analyse univariée
# ================================================================
num_cols = ['score','weighted_score','completion_rate','drop_rate','members','popularity','year','synopsis_length']
for col in num_cols:
    if col in df.columns:
        sns.histplot(df[col], bins=30, kde=True)
        plt.title(f"Distribution de {col}")
        plt.savefig(f"{REPORTS_DIR}/figures/dist_{col}.png", bbox_inches="tight")
        plt.close()  # close figure to free memory (and avoid duplicate displays with Agg)


In [ ]:
# 🎭 Analyse des genres et types
if "genres" in df.columns:
    top_genres = df["genres"].dropna().str.split(",").explode().str.strip().value_counts().head(20)
    top_genres.plot(kind="bar", color="teal")
    plt.title("Top 20 genres les plus fréquents")
    plt.ylabel("Nombre d'animes")
    plt.savefig(f"{REPORTS_DIR}/figures/top_genres.png", bbox_inches="tight")
    plt.show()

if "type" in df.columns:
    sns.countplot(data=df, y="type", order=df["type"].value_counts().index)
    plt.title("Distribution par type d'anime")
    plt.savefig(f"{REPORTS_DIR}/figures/type_distribution.png", bbox_inches="tight")
    plt.show()


C:\Users\isabe\AppData\Local\Temp\ipykernel_9000\3859258791.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\isabe\AppData\Local\Temp\ipykernel_9000\3859258791.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# ================================================================
# 🔗 4. Corrélations entre variables numériques
# ================================================================
corr_cols = [c for c in num_cols if c in df.columns]
corr = df[corr_cols].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Corrélations entre variables numériques")
plt.savefig(f"{REPORTS_DIR}/figures/correlation_heatmap.png", bbox_inches="tight")
plt.show()

# Exemples ciblés de relations
sns.scatterplot(data=df, x="score", y="completion_rate", alpha=0.5)
plt.title("Score vs Completion Rate")
plt.savefig(f"{REPORTS_DIR}/figures/score_vs_completion.png", bbox_inches="tight")
plt.show()


C:\Users\isabe\AppData\Local\Temp\ipykernel_9000\802044014.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\isabe\AppData\Local\Temp\ipykernel_9000\802044014.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# ================================================================
# 👥 5. Analyse du comportement utilisateur
# ================================================================
if user_favs is not None:
    user_activity = user_favs.groupby("username").size()
    anime_popularity = user_favs.groupby("mal_id").size()

    sns.histplot(user_activity, bins=30)
    plt.title("Distribution du nombre de favoris par utilisateur")
    plt.xlabel("Nombre de favoris")
    plt.ylabel("Nombre d'utilisateurs")
    plt.savefig(f"{REPORTS_DIR}/figures/user_activity.png", bbox_inches="tight")
    plt.show()

    top_animes = anime_popularity.sort_values(ascending=False).head(20)
    top_animes.plot(kind="bar", color="orange")
    plt.title("Top 20 animes les plus ajoutés en favoris")
    plt.ylabel("Nombre de favoris")
    plt.savefig(f"{REPORTS_DIR}/figures/top_favoris.png", bbox_inches="tight")
    plt.show()

    print(f"Nb moyen de favoris par utilisateur : {user_activity.mean():.2f}")
    print(f"Nb moyen de favoris par anime : {anime_popularity.mean():.2f}")
else:
    print("⚠️ user_favs non disponible pour cette exploration (aucun fichier trouvé).")


⚠️ user_favs non disponible pour cette exploration (aucun fichier trouvé).


In [ ]:
# ================================================================
# 📈 6. Tendances et insights clés
# ================================================================
print("🎯 INSIGHTS PRÉLIMINAIRES")

if 'completion_rate' in df.columns and 'score' in df.columns:
    corr_val = df['completion_rate'].corr(df['score'])
    print(f"➡️ Corrélation score ↔ completion_rate : {corr_val:.2f}")
    if corr_val > 0.5:
        print("   → Les animes bien notés ont tendance à être terminés plus souvent.")
    elif corr_val < 0.2:
        print("   → Le score n’explique que faiblement le taux de complétion.")

if 'members' in df.columns and 'popularity' in df.columns:
    corr_pop = df['members'].corr(df['popularity'])
    print(f"➡️ Corrélation members ↔ popularity : {corr_pop:.2f}")
    if corr_pop < -0.7:
        print("   → Popularity est une métrique inverse du nombre de membres (confirmé).")

if 'year' in df.columns:
    trend = df.groupby('year')['score'].mean().dropna()
    trend.plot(title="Évolution moyenne du score par année")
    plt.savefig(f"{REPORTS_DIR}/figures/trend_score_year.png", bbox_inches="tight")
    plt.show()


🎯 INSIGHTS PRÉLIMINAIRES
➡️ Corrélation score ↔ completion_rate : 0.12
   → Le score n’explique que faiblement le taux de complétion.
➡️ Corrélation members ↔ popularity : -0.59


C:\Users\isabe\AppData\Local\Temp\ipykernel_9000\3061170407.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# ================================================================
# 🧾 7. Rapport automatisé (optionnel)
# ================================================================
try:
    profile = ProfileReport(df, title="Anime Hybrid Dataset EDA", minimal=True)
    profile.to_file(os.path.join(REPORTS_DIR, "anime_eda_report.html"))
    print("💾 Rapport EDA sauvegardé dans /reports/anime_eda_report.html")
except Exception as e:
    print("⚠️ Impossible de générer le rapport Profiling :", e)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:02<00:00,  5.48it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Rapport EDA sauvegardé dans /reports/anime_eda_report.html


In [ ]:
# ================================================================
# 🧭 8. Conclusions & axes pour la modélisation
# ================================================================
print("""
📌 Conclusions de l’exploration :
───────────────────────────────
- Les distributions de scores et de completion_rate indiquent une base biaisée vers les animes populaires.
- Les genres et types dominants (Action, Comedy, Drama, TV) doivent être équilibrés ou pondérés.
- Corrélation notable entre engagement (completion_rate) et score → utile comme feature de qualité.
- Variables 'members' et 'popularity' très corrélées → à fusionner ou normaliser.
- synopsis_length variable selon le type → indicateur indirect de profondeur narrative.

🚀 Recommandations pour le prochain notebook (02_feature_engineering.ipynb) :
- TF-IDF ou Sentence Embeddings sur synopsis (feature textuelle principale)
- One-hot / Multi-label encoding sur genres, themes, type
- Normalisation MinMax sur scores, rates et members
- Construction d’un vecteur hybride : contenu (TF-IDF) + stats (numériques normalisées)
""")



📌 Conclusions de l’exploration :
───────────────────────────────
- Les distributions de scores et de completion_rate indiquent une base biaisée vers les animes populaires.
- Les genres et types dominants (Action, Comedy, Drama, TV) doivent être équilibrés ou pondérés.
- Corrélation notable entre engagement (completion_rate) et score → utile comme feature de qualité.
- Variables 'members' et 'popularity' très corrélées → à fusionner ou normaliser.
- synopsis_length variable selon le type → indicateur indirect de profondeur narrative.

🚀 Recommandations pour le prochain notebook (02_feature_engineering.ipynb) :
- TF-IDF ou Sentence Embeddings sur synopsis (feature textuelle principale)
- One-hot / Multi-label encoding sur genres, themes, type
- Normalisation MinMax sur scores, rates et members
- Construction d’un vecteur hybride : contenu (TF-IDF) + stats (numériques normalisées)

